---
title: Datasets II
subtitle: Work with Satellite Images and Other Multi-dimensional Data
authors:
  - name: Ian Carroll
    affiliations:
      - University of Maryland Baltimore County
      - NASA Goddard Space Flight Center
  - name: Rachel Wegener
    affiliations:
      - University of Maryland College Park
github: nasa-sarp/lesson-datasets-ii-east
---

::::{grid}

:::{card}
:header: Context 🤔
Gridded data, raster, and array are all descriptors for data
that doen't easilly fit in a spreadsheet.
:::

:::{card}
:header: Outcome 🎓
Exploring a NetCDF file that contains data from a remote sensing platform will take just a few lines
of Python.
:::

:::{card}
:header: Skills 🤓
Encountering an error won't be a roadblock; you'll use it to increase understanding!
:::
::::

In [ ]:
from pathlib import Path
import warnings
import logging

import xarray
import numpy
import pandas
import geopandas
import rioxarray
import hvplot.pandas
import hvplot.xarray
import holoviews as hv

DATADIR = Path('/efs/sarp/data/rawdata_readonly')
options = xarray.set_options(display_style='text')
hv.opts.defaults(
    hv.opts.Curve(active_tools=[], toolbar=None),
    hv.opts.Scatter(active_tools=[], toolbar=None),
    hv.opts.Image(active_tools=[]),
    hv.opts.Points(active_tools=[]),
    hv.opts.Layout(toolbar=None),
)
warnings.simplefilter("ignore")
logging.captureWarnings(True)

In [ ]:
hv.extension('bokeh')

## Data in Labelled Arrays

Python's built-in `list` data structure provides a basic, ultimately too basic, way to create multi-dimensional data arrays. Each level of nesting increases the number of dimensions: a scalar such as `42` is a zero-dimensional array, while `[42]`, `[[42]]`, and `[[[42]]]` are technically 1, 2, and 3 dimensional arrays. When working with multi-dimensional data, thinking about "levels of nesting" is more general than thinking about "rows" and "columns".

NumPy and XArray provide enhancements that make data processing easier to code and faster to run. This section introduces those packages along with the vocabulary they have adopted from pre-existing data conventions.

:::{seealso} Dimension
Dimensions are the axes along which individual data values are stacked in an array and usually refer to a physical location in space or time (e.g. depth, latitude, year, or date).

A 1D (one-dimensional) array, like a time-series, has a single axis. A 2D (two-dimensional) array, like a heat-map, has two axes. Dimensions have three properties: name, size, and position.
:::

A vertical profile of water temperature has a single dimension.

In [ ]:
temperature = [
    18.7,
    18.3,
    17.8,
    15.2,
    14.6,
    14.2,
    14.3,
]
temperature[3]

Multiple profiles from sequential times stack together on a second dimension. Note that the *second* dimension is the *second* level of nesting.

In [ ]:
temperature = [
    [18.7, 18.9, 18.4],
    [18.3, 18.4, 18.2],
    [17.8, 17.5, 18.1],
    [15.2, 14.9, 17.8],
    [14.6, 14.6, 17.5],
    [14.2, 14.1, 17.3],
    [14.3, 14.1, 17.1],
]
temperature[4][2]

The NumPy package makes this nested list into an array, with fixed data type, that is aware of its dimensionality. These are two critical differences between a `numpy.ndarray` and a `list`.

In [ ]:
temperature = numpy.array(temperature, dtype=float)
temperature.shape

:::{seealso} Shape
The shape of an array is the ordered sequence of dimension sizes. Shape depends on two of the three properties of each dimension: size and position.
:::

The XArray package converts a `numpy.ndarray` to an array with named dimensions.

In [ ]:
profile = xarray.DataArray(
    temperature,
    dims=('depth', 'time'),
    name='temperature',
)
profile

:::{seealso} Coordinate
Coordinates are special data arrays that provide values, often locations in space or time, for one of the dimensions.

A coordinate is a 1D data array with two additional requirements: a name equal to the name of its dimension, a length equal to the size of its dimension.
:::

Adding both depth and time arrays enriches the temperature data with coordinates.

In [ ]:
profile['depth'] = ('depth', [0.0, 2.5, 5.0, 7.5, 10.0, 12.5, 15.0])
profile

In [ ]:
profile['time'] = ('time', pandas.date_range('2023-06-14', periods=2))
profile

Reading the error message points to a mistake in the previous cell: the number of periods was at fault.

In [ ]:
profile['time'] = ('time', pandas.date_range('2023-06-14', periods=3))
profile

:::{note} Review
XArray maintains all three properties of each dimension (name, size, and position) and only lists under "Coordinates" those arrays meeting the requirements (name equal to dimension name, length equal to dimension size).
:::

With hvPlot's plugin for XArray, labels you include on your data array apear automatically in plots.

In [ ]:
(
    profile
    .hvplot(y='depth', by='time', aspect=0.5)
    .opts(invert_yaxis=True)
)

## Array Operations

Arrays make bulk operations possible, which makes code easier to write and faster to run. This section introduces a few important operations for when you need to drill-down on part of an array, make some calculations, or combine arrays.

### Selecting

:::{tip} xarray.DataArray.sel
Returns a new xarray.DataArray whose data is given by selecting index labels along the specified dimension(s).

Supply a dictionary with dimension names as keys and values that can be looked up in the corresponding coordinate.
:::

Pass a scalar in a dictionary with dimension names as keys to subset from an array at a single location on the given dimension. The data array returned has one less dimension.

In [ ]:
profile.sel({'depth': 7.5})

Pass a `slice` with starting and ending bounds (or use `None` for unbounded) to return a data array confined to the given coordinate range.

In [ ]:
profile.sel({'depth': slice(None, 7.5)})

:::{tip} xarray.DataArray.where(cond, other=nan)
Returns a new xarray.DataArray with the original data where `cond` is `True`, elsewhere its data comes from `other`.
:::

Masking is another way of selecting from an array, but without subsetting. Masking requires a boolean array (an array of `True` or `False` values) as an argument.

In [ ]:
profile['time'].dt.dayofweek == 2

In [ ]:
profile.where(
    cond=profile['time'].dt.dayofweek == 2,
)

### Transforming

Array's have several methods, including calculating averages, that accept the name of the dimension(s) to eliminate from the array by some calculation.

In [ ]:
t_avg = profile.mean(dim='depth')
t_avg

XArray provides a container, the `xarray.Dataset`, that holds one or more arrays. It will be convenient for demonstrating array transformations that produce arrays having shared coordinates.

In [ ]:
station = profile.to_dataset()
station

Algebra works on arrays by performing the operation (`+`, `-`, `*`, `/`, `%`, `//`, or `**`) pair-wise on each element. When arrays do not have the same dimensions, XArray uses their labels to make the arrays compatible.

In [ ]:
station['wind'] = (0.02 * t_avg) ** -2
station

In [ ]:
station['density'] = (
    19.6
    - 12.8 * numpy.log(station['temperature'])
    + 2.3 * station['wind']
)
station

The plots generated with hvPlot can be combined into subplots by concatenating (with `+`) them together.

In [ ]:
kwargs = {'y': 'depth', 'by': 'time', 'aspect': 0.5}
t_plt = station['temperature'].hvplot(**kwargs)
s_plt = station['density'].hvplot(**kwargs)
(
    t_plt.opts(invert_yaxis=True, show_legend=False)
    + s_plt.opts(invert_yaxis=True)
)

:::{attention} Exercise 1
Open the "practice.ipynb" notebook and tackle the first exercise.
:::

### Combining

Data arrays that come from different sources can be joined wherever the coordinates match.

In [ ]:
chla = xarray.DataArray(
    data=[
        [1.3, 1.1, 0.5],
        [0.2, 0.2, 0.4],
    ],
    coords={
        'depth': [0.0, 10.0],
        'time': station['time'],
    },
    name='chla',
)
chla

:::{tip} xarray.merge(objects)
Returns a new xarray.Dataset with arrays and coordinates from the input objects, combining coordinates and resizing arrays as needed. The result is highly configurable with additional arguments.
:::

In [ ]:
station = xarray.merge((station, chla))
station['chla']

## Files for Array Data

Multi-dimensional arrays containing earth observations tend to be large datasets, too large to store in a text file. While text-based data formats like CSV, JSON, and even XLSX, are convenient, binary files pack data more densely and work faster.

### Science Data Files

XArray can read and write multiple binary formats natively and many more formats can be read into the XArray objects using the rioxarray package. Natively, XArray will read the Hierarchical Data Format (HDF), the closely related NetCDF format, and Zarr. Common file extensions for these formats are `.h5`, `.nc`, `.nc4`, or `.zarr`.

:::{note} HDF5 and NetCDF4
HDF version 5 is part of NetCDF version 4: NetCDF changed in version 4 to rely on HDF version 5 under the hood. Prior versions of both formats are not as well supported but may be encountered in the wild.
:::

In [ ]:
file = (
    DATADIR
    / 'oco-3-co2-data'
    / 'oco3_LtCO2_200228_B10400Br_220317235859s.nc4'
)
oco3 = xarray.open_dataset(file, chunks={})
oco3['xco2']

:::{seealso} Attributes
File formats used for array data also store metadata that XArray reads into an `attrs` dictionary. The `xco2` data array has attributes that follow the CF Conventions for metadata, including the all important `units` attribute. The dataset has attributes that are not specific to a variable.
:::

In [ ]:
oco3.attrs['Platform']

Opening a dataset like this for the first time demands some visual exploration (always plot your data!). Subsetting the data by the date variable keeps it small for this tutorial.

In [ ]:
oco3 = oco3.where((oco3['date'].isel({'epoch_dimension': 3}) == 15).compute(), drop=True)

The dataset includes date, latitude, and longitude values along the same coordinate, `sounding_id`. This is a not an image with a latitude and longitude dimension; instead, these data are a flight trajectory and more amenable to a scatter plot. The date variable is non standard, it would take a few steps to create a standard datetime variable to include in any plots.

In [ ]:
oco3_pandas = oco3[['latitude', 'longitude', 'xco2']].to_dataframe()
oco3_pandas.hvplot.scatter(x='longitude', y='latitude', color='xco2')

:::{attention} Exercise 2
Open the "practice.ipynb" notebook and tackle the second exercise.
:::

### Geospatial Data Files

Dedicated packages are available for working with data arrays that have geospatial coordinates, such as the latitude and longitude values in the OCO3 dataset. GeoPandas, an extension to Pandas developed for geospatial data manipulation, is one example. Casting the OCO3 data to a GeoPandas data frame provides some immediate enhancements to the scatter plot.

In [ ]:
oco3_vector = geopandas.GeoDataFrame(
    data={'xco2': oco3['xco2']},
    geometry=geopandas.points_from_xy(
        oco3['longitude'],
        oco3['latitude'],
    ),
)
oco3_vector['geometry']

In [ ]:
oco3_vector.hvplot.points(color='xco2', coastline=True)

::::{grid} 1 1 2 2

:::{seealso} Vector Geospatial Data
A geospatial "vector" dataset is one-dimensional data with a coordinate whose data type is a compound data type (i.e. not a scalar integer or float) that represents either a point, a linestring, a polygon, or some combination of these. The `oco3_vector` data frame has a variable with such a `dtype`, and by convention this variable is also named "geometry".
:::

:::{seealso} Raster Geospatial Data
A data array with two dimensions corresponding either to latitude and longitude or to projected "x" and "y" coordinates, is called a "raster". What makes a raster different from any other two-dimensional data array is required metadata declaring how to interpret the coordinates by rigid specification of the Coordinate Reference System (CRS).
:::

::::

There are both text-based (e.g. `.geojson`) and binary (e.g. `.shp`) file formats specific to vector data. Text-based formats, as usual, are convenient for small datasets. Clipping a collection of geometries by a bounding box is another enhancement GeoPandas brings to data frames, which drastically shrinks the global OCO3 dataset before saving it as GeoJSON.

In [ ]:
oco3_vector = oco3_vector.cx[-82:-72, 42:36]
oco3_vector.to_file('oco3_vector.geojson')

The rioxarray package extends XArray to read multiple binary formats for rasters leveraging all the utility of the Geospatial Data Abstraction Library (GDAL). A common file extension for raster data is `.tif`, but not all `.tif` files include a CRS. Some NetCDF files are recognized by GDAL as containing rasters because they include the CRS in an attribute (e.g. the attribute `crs_wkt`).

:::{tip} rasterio.Env
For GDAL to work correctly with file objects on AWS, the rasterio package implements a Python context manager that sets all the relevant GDAL configuration for remote data access.
:::

Large data collections are best left where they lie, especially when they lie on Amazon's S3. 

In [ ]:
!pip install --user git+https://github.com/NASA-SARP/sarp-east-toolkit.git

In [ ]:
from sarp_east_toolkit import earthdata_rio

rio_env = earthdata_rio('ornldaac')

In [ ]:
fileobj = (
    's3://ornl-cumulus-prod-protected/'
    'gedi/GEDI_L4B_Gridded_Biomass/data/'
    'GEDI04_B_MW019MW138_02_002_05_R01000M_MU.tif'
)
with rio_env as env:
    raster = xarray.open_dataset(fileobj, engine='rasterio', chunks={})
raster

The band_data variable requires {eval}`f"{raster['band_data'].data.nbytes / 1024**3:.2} GB"` of memory to load, which despite the speed of S3 will still be slower than reading from local storage. Fortunately, it is possible to read only the subset of the file corresponding to a selection.

In [ ]:
raster['spatial_ref']

In [ ]:
crs = raster.rio.crs
crs

In [ ]:
oco3_vector_prj = oco3_vector.to_crs(crs)

In fact, both raster and vector geospatial arrays need a defined CRS. The GeoJSON format skips over this added complexity by declaring that all GeoJSON data must use the CRS known as "EPSG:4326". These [EPSG codes](https://spatialreference.org/) exist because defining exactly how a location in, on, or above Earth is assigned coordinates is very complicated!

In [ ]:
oco3_vector_prj = oco3_vector.set_crs(epsg=4326).to_crs(crs)
oco3_vector_prj.crs

With the vector and raster data sets in the same CRS, it is possible to use the total geospatial extent of the OCO3 data to clip the GEDI data. "Clipping" is just geospatial jargon for selecting a slice along the two coordinates of a data array.

In [ ]:
bbox = oco3_vector_prj.total_bounds
bbox

In [ ]:
biomass_density = (
    raster['band_data']
    .sel({'band': 1})
    .rio.clip_box(*bbox)
)
biomass_density.data.nbytes

The resulting raster is small enough now to load and visualize together with the vector data.

In [ ]:
(
    biomass_density.hvplot.image(crs=crs.to_epsg())
    * oco3_vector_prj.hvplot.points(color='orange', crs=crs.to_epsg())
)

:::{hint} Surprise!
What have we learned after all that work? GEDI and OCO3 must be on the same platform!
:::

# Next Steps

- This introduction to XArray leaves a lot out. A few suggestions to [read up](https://pypi.org) on:
    - `xarray.open_mfdataset` for reading a dataset stored in multiple files
    - `xarray.Dataset.groupby` or `xarray.DataArray.groupby` for complex array operations
    - the [xarray-datatree](https://xarray-datatree.readthedocs.io/en/latest/) for reading files with "groups" of data arrays (i.e. the "hierarchical" part of HDF).

- Coordinate Reference Systems, and switching between them, is a really challenging aspect of working with geospatial data. Dive into learning about CRS at [pygis.io](https://pygis.io/docs/d_crs_what_is_it.html).

- Needing help is to be expected, but getting help is more likely with good questions:
  - read the documentation
  - write a Minimal Reproducible Example ([MRE](https://stackoverflow.com/help/minimal-reproducible-example))
  - post your question and MRE in a suitable forum (not a package issue tracker)

:::{attention} Exercise 3
Open the "practice.ipynb" notebook and tackle the final exercise.
:::

## Closing Poll

The [closing poll](https://PollEv.com/clickable_images/WHersUlQe5SsG68KOHyfL/respond) which is, as all the others are, anonymous.

:::{danger} Shutdown
Please shut down your server! (File > Hub Control Panel)
:::